In [1]:
import os
import numpy as np
from tensorflow.keras.applications import EfficientNetB0

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data_dir = '/Users/saikarthik/Downloads/untitled folder/parkinsons_dataset'


In [3]:
img_height, img_width = 224, 224 
batch_size = 32


In [4]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


In [5]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

Found 665 images belonging to 2 classes.


In [6]:

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 166 images belonging to 2 classes.


In [7]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

2025-04-02 12:48:13.971306: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-02 12:48:13.971340: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-02 12:48:13.971347: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-02 12:48:13.971544: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-02 12:48:13.971557: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


16705208/16705208 [==============================] - 5s 0us/step


In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
)

Epoch 1/100


2025-04-02 12:48:24.120116: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - ETA: 0s - loss: 0.6425 - accuracy: 0.6722

2025-04-02 12:48:30.690355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 11s 341ms/step - loss: 0.6425 - accuracy: 0.6722 - val_loss: 0.5906 - val_accuracy: 0.7349
Epoch 2/100
21/21 [==============================] - 5s 239ms/step - loss: 0.5883 - accuracy: 0.7338 - val_loss: 0.5807 - val_accuracy: 0.7349
Epoch 3/100
21/21 [==============================] - 5s 236ms/step - loss: 0.5855 - accuracy: 0.7338 - val_loss: 0.5791 - val_accuracy: 0.7349
Epoch 4/100
21/21 [==============================] - 5s 237ms/step - loss: 0.5885 - accuracy: 0.7338 - val_loss: 0.5804 - val_accuracy: 0.7349
Epoch 5/100
21/21 [==============================] - 5s 240ms/step - loss: 0.5891 - accuracy: 0.7338 - val_loss: 0.5789 - val_accuracy: 0.7349
Epoch 6/100
21/21 [==============================] - 5s 236ms/step - loss: 0.5871 - accuracy: 0.7338 - val_loss: 0.5787 - val_accuracy: 0.7349
Epoch 7/100
21/21 [==============================] - 5s 235ms/step - loss: 0.5858 - accuracy: 0.7338 - val_loss: 0.5790 - val_accuracy: 0.7349
Epoch 8/10

In [14]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

validation_generator.reset()
predictions = (model.predict(validation_generator) > 0.5).astype("int64")
true_classes = validation_generator.classes

print("\nClassification Report:")
print(classification_report(true_classes, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, predictions))

6/6 [==============================] - 1s 154ms/step - loss: 0.5799 - accuracy: 0.7349
Validation Accuracy: 73.49%
6/6 [==============================] - 1s 164ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.73      1.00      0.85       122
           1       0.00      0.00      0.00        44

    accuracy                           0.73       166
   macro avg       0.37      0.50      0.42       166
weighted avg       0.54      0.73      0.62       166


Confusion Matrix:
[[122   0]
 [ 44   0]]


/Users/saikarthik/tensorflow-test/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/saikarthik/tensorflow-test/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/saikarthik/tensorflow-test/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,